In [ ]:
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
from trl import DPOConfig, DPOTrainer

In [ ]:
# -----------------------------
# 0) データ（最小例）
#   実運用ではここを自前の preference dataset に置き換えます
#   形式: prompt / chosen / rejected
# -----------------------------
data = [
    {
        "prompt": "<|system|>ユーザの発言を平易化し，さらに書き言葉から話し言葉に変換してください。</s><|user|>千葉工業大学は千葉県習志野市にある工業大学である。</s>",
        "chosen": "千葉工大は、千葉の習志野にある工業系の大学なんだ。",
        "rejected": "千葉工業大学は、千葉の習志野にある工業系の大学だよ。簡単に言うと、工学を学ぶための大学なんだ。場所は、千葉県の習志野ってところにあって、分かりやすい場所にあるよ",
    },
]
train_ds = Dataset.from_list(data)

In [ ]:
# -----------------------------
# 1) モデルとトークナイザ
# -----------------------------
# model_name = "gpt2"  # デモ用。実際は任意の CausalLM に置換
model_name = "sbintuitions/sarashina2.2-3b-instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # DPO では padding が必要になることが多い

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)

# reference model（π_ref）：固定。policy と同一初期値から開始するのが典型
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=base_model.dtype,
)

In [ ]:
type(base_model)

In [ ]:
# -----------------------------
# 2) PEFT (LoRA) を policy 側だけに適用
#   - ref_model は固定のまま（LoRA すら載せないのが「最小」）
# -----------------------------
lora_config = LoraConfig(
    r=8,                     # 8 or 16 がまずは無難
    lora_alpha=16,           # 通常 r の 2 倍
    lora_dropout=0.05,       # DPO では 0.0〜0.1
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
    ],
)

model = get_peft_model(base_model, lora_config)

In [ ]:
# -----------------------------
# 3) DPO 設定（β、バッチ等）
#   - beta が 2.5 の β
# -----------------------------
dpo_args = DPOConfig(
    output_dir="./dpo_lora_out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    num_train_epochs=1,
    beta=0.1,
    logging_steps=1,
    save_steps=50,
    fp16=torch.cuda.is_available(),
    # 安定性のために max_length / max_prompt_length を入れるのが一般的
    max_length=512,
    max_prompt_length=256,
)


In [ ]:
# -----------------------------
# 4) Trainer
#   - 内部で log πθ(chosen|prompt), log πθ(rejected|prompt)
#          log πref(chosen|prompt), log πref(rejected|prompt)
#     を計算して 2.5 の損失を作ります
# -----------------------------
trainer = DPOTrainer(
    model=model,                 # πθ（LoRA付き）
    ref_model=ref_model,         # π_ref（固定）
    args=dpo_args,
    train_dataset=train_ds,
)
